# Overview

This notebook analyzes the use of levenberg-marquardt for IK refinement

In [ ]:
%load_ext autoreload
%autoreload 2
# %load_ext wurlitzer
# import os
# os.chdir("../")
# print("Current working directory:", os.getcwd())

In [ ]:

import torch
import numpy as np
from jrl.utils import set_seed, make_text_green_or_red
from jrl.math_utils import geodesic_distance_between_quaternions
import matplotlib.pyplot as plt

from ikflow.model_loading import get_ik_solver


torch.set_printoptions(linewidth=200, precision=6, sci_mode=False)

In [ ]:
MODEL_NAME = "panda__full__lp191_5.25m"
POS_ERROR_THRESHOLD = 0.001
ROT_ERROR_THRESHOLD = 0.01

# n_solutions = 40
n_solutions = 200

ikflow_solver, _ = get_ik_solver(MODEL_NAME)
robot = ikflow_solver.robot

set_seed()
_, target_poses = ikflow_solver.robot.sample_joint_angles_and_poses(n_solutions, only_non_self_colliding=True, tqdm_enabled=False)
target_poses = torch.tensor(target_poses, dtype=torch.float32, device="cuda:0")

In [ ]:
def check_converged(robot, qs, target_poses, i):
    # check error
    pose_realized = robot.forward_kinematics_batch(qs)
    pos_errors = torch.norm(pose_realized[:, 0:3] - target_poses[:, 0:3], dim=1)
    rot_errors = geodesic_distance_between_quaternions(target_poses[:, 3:], pose_realized[:, 3:])
    #
    converged = pos_errors.max().item() < POS_ERROR_THRESHOLD and rot_errors.max().item() < ROT_ERROR_THRESHOLD
    # print("solve_lma", i, converged, round(pos_errors.max().item(), 6), "\t", round(rot_errors.max().item(), 6), "\t", pos_errors.data)
    # print("solve_lma", i, converged, round(pos_errors.max().item(), 6), "\t", round(rot_errors.max().item(), 6),)
    #
    # converged = pos_errors.mean().item() < POS_ERROR_THRESHOLD and rot_errors.mean().item() < ROT_ERROR_THRESHOLD
    # print("solve_lma", i, converged, round(pos_errors.mean().item(), 6), "\t", round(rot_errors.mean().item(), 6), "\t", pos_errors.data, rot_errors.data)
    return converged, pos_errors, rot_errors


def plot_errors(all_pos_errors, all_rot_errors):
    fig, (axl, axr) = plt.subplots(1, 2, figsize=(18, 8))
    fig.suptitle(f"Levenberg-Marquardt IK Convergence")

    axl.set_title(f"Positional errors")
    axl.grid(alpha=0.2)
    axl.set_xlabel("iteration")
    axl.set_ylabel("error (m)")

    axr.set_title(f"Rotational errors")
    axr.grid(alpha=0.2)
    axr.set_xlabel("iteration")
    axr.set_ylabel("error (rad)")

    n = all_pos_errors[0].shape[0]
    xs = np.arange(n) * 5
    axl.plot([0, xs[-1]], [POS_ERROR_THRESHOLD, POS_ERROR_THRESHOLD], label="threshold", color="green")
    axr.plot([0, xs[-1]], [ROT_ERROR_THRESHOLD, ROT_ERROR_THRESHOLD], label="threshold", color="green")
    for i, (pos_errors, rot_errors) in enumerate(zip(all_pos_errors, all_rot_errors)):
        axl.scatter(xs, pos_errors.cpu().numpy(), label=f"iteration {i}", s=15)
        axr.scatter(xs, rot_errors.cpu().numpy(), label=f"iteration {i}", s=15)
    # plt.savefig(f"lma_errors_{i}.pdf", bbox_inches="tight")
    # axl.legend()
    axr.legend()
    plt.show()


set_seed(0)
qs = ikflow_solver.solve_n_poses(ys=target_poses)
# set_seed(10) # n_solutions = 40
# qs = ikflow_solver.solve_n_poses(ys=target_poses)

converged = False
i = 0
all_pos_errors = []
all_rot_errors = []

_, pos_errors, rot_errors = check_converged(robot, qs, target_poses, "init")
all_pos_errors.append(pos_errors)
all_rot_errors.append(rot_errors)

alphas = torch.ones((n_solutions, 1), dtype=torch.float32, device="cuda:0")

thresh = 0.015
above_thresh_scale = 0.5
alphas[pos_errors > thresh] = above_thresh_scale

while not converged and i < 100:

    qs = robot.inverse_kinematics_single_step_levenburg_marquardt(target_poses, qs)
    # qs = robot.inverse_kinematics_single_step_levenburg_marquardt(target_poses, qs, alpha=0.1)           # 53 iterations
    # qs = robot.inverse_kinematics_single_step_levenburg_marquardt(target_poses, qs, alpha=0.25)          # 20 iterations
    # qs = robot.inverse_kinematics_single_step_levenburg_marquardt(target_poses, qs, alpha=0.75)          # 27 iterations
    # qs = robot.inverse_kinematics_single_step_levenburg_marquardt(target_poses, qs, alpha=0.5)         # 10 iterations

    # qs = robot.inverse_kinematics_single_step_levenburg_marquardt(target_poses, qs, alphas=alphas)

    converged, pos_errors, rot_errors = check_converged(robot, qs, target_poses, i)
    all_pos_errors.append(pos_errors)
    all_rot_errors.append(rot_errors)


    # 
    alphas[pos_errors > thresh] = above_thresh_scale
    alphas[pos_errors < thresh] = 1.0


    # print(i, converged)
    i += 1


print()
if not converged:
    print(make_text_green_or_red(f"Failed to converge after {i} iterations", False))
else:
    print(make_text_green_or_red(f"Converged after {i} iterations", True))

plot_errors(all_pos_errors, all_rot_errors)